In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import warnings
warnings.filterwarnings('ignore')
torch.cuda.empty_cache() 

In [ ]:
PATH="../input/100-bird-species/birds"
train_dir = os.path.join(PATH, 'train')
valid_dir = os.path.join(PATH, 'valid')
test_dir = os.path.join(PATH, 'test')

In [ ]:
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, Dataset
batch_size = 64
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(227),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])


In [ ]:
train_data = torchvision.datasets.ImageFolder(
        root=train_dir,
        transform= transform
    )
valid_data = torchvision.datasets.ImageFolder(
        root=valid_dir,
        transform= transform
    )
test_data = torchvision.datasets.ImageFolder(
        root=test_dir,
        transform= transform
    )

train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
valid_dl = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=0)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)


In [ ]:
classes = train_data.classes


In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    plt.imshow(np.transpose(img, (1, 2, 0)))
dataiter = iter(train_dl)
images, labels = dataiter.next()
images = images.numpy()
fig = plt.figure(figsize=(25,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks = [], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self, num_classes=275):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3,96,11,4)
        self.conv2 = nn.Conv2d(96,256,5,padding=2,groups=2)
        self.conv3 = nn.Conv2d(256,384,3,padding=1)
        self.conv4 = nn.Conv2d(384,384,3,padding=1, groups=2)
        self.conv5 = nn.Conv2d(384,256,3,padding=1, groups=2)
        self.fc1 = nn.Linear(256*6*6,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,num_classes)
 
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(F.relu(self.conv5(x)),(2,2))
        x = x.view(x.size(0),256*6*6)
        x = F.dropout(F.relu(self.fc1(x)),p=0.5)
        x = F.dropout(F.relu(self.fc2(x)),p=0.5)
        x = self.fc3(x)
        return x
device = torch.device("cuda")
print(device)
 
net = Model()
net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.1)
 
for epoch in range(20):
 
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        inputs, lables = data
        inputs = inputs.to(device)
        lables = lables.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, lables)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d,%5d] loss: %.5f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


In [ ]:

correct_predictions = 0
total_predictions = 0
for inputs, labels in valid_dl:
    if torch.cuda.is_available():
        inputs, labels = inputs.cuda(), labels.cuda() 
        outputs = net(inputs)
        prediction = outputs.argmax(1)
        correct_predictions += prediction.eq(labels.data).sum().item()
        total_predictions += labels.size(0)
print('Validation_Accuracy: %.2f %%' % (100 * (correct_predictions/total_predictions)))

In [ ]:

correct_predictions = 0
total_predictions = 0
for inputs, labels in test_dl:
    if torch.cuda.is_available():
           inputs, labels = inputs.cuda(), labels.cuda() 
           outputs = net(inputs)
           prediction = outputs.argmax(1)
           correct_predictions += prediction.eq(labels.data).sum().item()
           total_predictions += labels.size(0)
print('Test_Accuracy: %.2f %%' % (100 * (correct_predictions/total_predictions)))